In [159]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import json

ASSUME_TZ = None
HORIZON_H = 3
LOOKBACK_L = 48
TEST_SAMPLES = 2000

FEATURES = ['price_pct_change', 'volume_pct_change']

In [160]:
model = tf.keras.models.load_model("./results/models/lstm.regression.keras")

In [161]:
eth_data = pd.read_csv('./paper_wallet_data/eth_data.csv')
eth_data = eth_data.dropna().reset_index(drop=True)
eth_data.head()

,Unnamed: 0,time,ETH_price,ETH_volume
0,12063,2016-06-09 15:00:00,14.69,333.920307
1,12064,2016-06-09 16:00:00,14.69,667.084306
2,12065,2016-06-09 17:00:00,14.57,355.767139
3,12066,2016-06-09 18:00:00,14.65,561.489527
4,12090,2016-06-10 18:00:00,14.10,1238.313305


In [162]:
eth_data['time'] = pd.to_datetime(eth_data['time'], utc=True)
if ASSUME_TZ is not None:
    eth_data['time'] = eth_data['time'].dt.tz_convert('UTC')

eth_data["hour"] = eth_data["time"].dt.hour

In [163]:
eth_data['price_pct_change'] = eth_data['ETH_price'].pct_change(1)   
eth_data['volume_pct_change'] = eth_data['ETH_volume'].pct_change(1)
eth_data = eth_data.dropna().reset_index(drop=True)
hours = eth_data['hour']
hours = np.array(hours, dtype=np.int32)   # force integers 0–23

y = eth_data['ETH_price'].pct_change(HORIZON_H)

In [164]:
#get ETH id
with open("paper_wallet_data/asset_to_id.json", "r") as f:
    asset_to_id = json.load(f)

# Example usage
eth_id = asset_to_id["ETH"]
print(f"ETH Id: {eth_id}")

ETH Id: 8


In [165]:
def make_sequences(df, L):
    X_list = []
    X = df[FEATURES].values
    X = X
    for t in range(L, len(df)):
            X_list.append(X[t-L:t,:])
    return np.stack(X_list[-TEST_SAMPLES:], dtype=np.float64)

X = make_sequences(eth_data, LOOKBACK_L)

In [166]:
A = np.full((TEST_SAMPLES,), eth_id, dtype=np.int32)

In [167]:
yhat = model.predict({"Returns": X, "Tickers": A})
print(yhat.shape)  # (1000, 1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
(2000, 1)


In [168]:
def compute_true_targets(prices, horizon=3, n_samples=TEST_SAMPLES):
    """
    Compute true forward % change targets for the last n_samples windows.
    
    prices: array-like of raw prices
    horizon: prediction horizon (steps ahead)
    n_samples: how many sequences we predicted
    seq_len: length of each input window (48 in your case)
    """
    prices = np.asarray(prices, dtype=np.float32)
    
    # forward return for every valid index
    forward = (prices[horizon:] - prices[:-horizon]) / prices[:-horizon]
    
    # slice the last n_samples, aligned with your X windows
    true_vals = forward[-n_samples:]
    
    return true_vals

prices = eth_data['ETH_price']
true_changes = compute_true_targets(prices, horizon=3, n_samples=TEST_SAMPLES)

In [169]:
# Metrics
mae = np.mean(np.abs(yhat - true_changes))
#corr = np.corrcoef(yhat, true_changes)[0,1]
directional_acc = np.mean(np.sign(yhat) == np.sign(true_changes))

print(f"MAE: {mae:.4f}, DirAcc: {directional_acc:.2%}")

MAE: 0.0097, DirAcc: 50.83%


In [170]:
y_count = np.sum(yhat < 0)
print(f"Predicted Up predictions: {len(yhat) - y_count} Down predictions: {y_count}")
true_count = np.sum(true_changes < 0)
print(f"Real Up predictions: {len(true_changes) - true_count} Down predictions: {true_count}")

Predicted Up predictions: 1216 Down predictions: 784
Real Up predictions: 1080 Down predictions: 920


In [171]:
correct = 0
for i in range(len(yhat)):
    if (yhat[i] < 0 and true_changes[i] < 0) or (yhat[i] > 0 and true_changes[i] > 0):
        correct += 1

print(f"Correct direction: {correct} out of {len(yhat)}")

Correct direction: 1577 out of 2000


In [172]:
yhat[:10]

array([[ 0.00681133],
       [ 0.00454094],
       [ 0.00303155],
       [ 0.00053786],
       [-0.00947612],
       [ 0.00019584],
       [ 0.01192352],
       [ 0.00754448],
       [ 0.00318329],
       [ 0.00036662]], dtype=float32)

In [173]:
true_changes[:10]

array([ 0.00749497,  0.00653031, -0.00061215, -0.00792378, -0.00490625,
        0.0074824 ,  0.01425597,  0.01175291, -0.00122018, -0.00087031],
      dtype=float32)